# Importing packages and loading data

In [119]:
#LOAD AND IMPORT PACKAGES AND DATA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cptac
import os

os.chdir('/Users/andreavelazquez/Desktop/School stuff/QBIO490/qbio_490_AndreaV/analysis_data')
cptac.download('Ovarian')

True

In [146]:
#PULL DATA
#clinical data
ov= cptac.Ovarian()
clinical_data= ov.get_clinical()

#protein data
protein_data= ov.get_proteomics()
protein_data.columns = protein_data.columns.get_level_values(0) 

#transcriptomic data
rna_data= ov.get_transcriptomics()

# Data preprocessing

In [147]:
#CLEANUP
#Change NaN values to 0 so we can get rid of them
rna_data = rna_data.replace(0, np.nan)
gene_na_mask = rna_data.isna().sum() == 0
rna_masked = rna_data.loc[:, gene_na_mask]
#logscale
rna_masked= np.log2(rna_masked)

#Cleanup proteins w no values in there
protein_na_mask = protein_data.isna().sum() == 0
proteins_masked = protein_data.loc[:, protein_na_mask]


# Consolidating gene list

In [123]:
#GENE LIST FROM EXPLORATORY DATA ANALYSIS WITH TCGA DATA IN R
top53_R= ['TP53', 'TTN', 'CSMD3', 'USH2A', 'NF1', 'RYR2', 
          'MUC16', 'HMCN1', 'FAT3', 'FLG2', 'MACF1', 'SI', 
          'AHNAK', 'MUC17', 'APOB', 'FLG', 'DNAH3', 'LRP1B', 
          'CDK12', 'DST', 'DNAH5', 'LRP1', 'BRCA1', 'SYNE2', 
          'FCGBP', 'SYNE1', 'TG', 'TOP2A', 'DNAH10', 'RB1',
          'RELN', 'COL6A3', 'DNAH8', 'MDN1', 'ADGRV1', 'HUWE1', 
          'MYH4', 'KMT2C', 'TENM1', 'TACC2', 'DYNC1H1', 'PKHD1', 
          'NEB', 'RYR1', 'LRP2', 'CSMD1', 'DMD', 'IGSF10', 
          'LRRK2', 'ZFHX4', 'MYH1', 'KMT2A', 'VPS13B' ]

In [124]:
#GENE LIST FROM LITERATURE REVIEW
lit_genes= ['CLDN3', 'CLDN4', 'HE4', 'CRIP1', 'APOE', 
            'APOJ', 'HLA-DRA', 'HLA-DRB', 'MUC1', 'BCAM', 
            'BUB1B', 'BUB1', 'TTK', 'CCNB1', 'BRCA1', 
            'BRCA2', 'CSMD3', 'MUCI16', 'APOB', 'DST', 
            'FAT3', 'HMCN1', 'RYR1','TP53','TTN', 
            'USH2A', 'AKT1', 'ARID1A', 'FBXW7', 'FGFR2', 
            'JAK1', 'KRAS', 'MLH1', 'MSH2', 'MSH6', 
            'NRAS', 'PIK3CA', 'PIK3R1', 'PIK3R2', 'PMS2', 
            'POLE', 'PPP2R1A', 'PTEN', 'RNF43', 'RPL22', 
            'SMARCA4', 'STK11', 'SPOP', 'FOXL2', 'CTNNB1', 
            'BRAF']

In [159]:
#Combine both lists into one without duplicates
final_list= helper(top53_R, lit_genes)

CLDN3  added to new list
CLDN4  added to new list
HE4  added to new list
CRIP1  added to new list
APOE  added to new list
APOJ  added to new list
HLA-DRA  added to new list
HLA-DRB  added to new list
MUC1  added to new list
BCAM  added to new list
BUB1B  added to new list
BUB1  added to new list
TTK  added to new list
CCNB1  added to new list
BRCA2  added to new list
MUCI16  added to new list
AKT1  added to new list
ARID1A  added to new list
FBXW7  added to new list
FGFR2  added to new list
JAK1  added to new list
KRAS  added to new list
MLH1  added to new list
MSH2  added to new list
MSH6  added to new list
NRAS  added to new list
PIK3CA  added to new list
PIK3R1  added to new list
PIK3R2  added to new list
PMS2  added to new list
POLE  added to new list
PPP2R1A  added to new list
PTEN  added to new list
RNF43  added to new list
RPL22  added to new list
SMARCA4  added to new list
STK11  added to new list
SPOP  added to new list
FOXL2  added to new list
CTNNB1  added to new list
BRAF  

In [163]:
#CHECK WE HAVE INFO FOR THOSE GENES/ARE IN DF
#1st element=is in df, 1nd element, not in df
in_proteins_df= inDF(final_list, proteins_masked)
not_in_proteins= in_proteins_df[1].copy()

#search for the remaining ones in rna df
in_rna_df= inDF(not_in_proteins, rna_masked)
in_rna_df

CSMD3  is not in the df
USH2A  is not in the df
RYR2  is not in the df
FAT3  is not in the df
FLG2  is not in the df
SI  is not in the df
MUC17  is not in the df
FLG  is not in the df
DNAH3  is not in the df
DNAH5  is not in the df
BRCA1  is not in the df
TG  is not in the df
DNAH10  is not in the df
RELN  is not in the df
DNAH8  is not in the df
ADGRV1  is not in the df
KMT2C  is not in the df
TENM1  is not in the df
PKHD1  is not in the df
RYR1  is not in the df
LRP2  is not in the df
CSMD1  is not in the df
IGSF10  is not in the df
LRRK2  is not in the df
ZFHX4  is not in the df
HE4  is not in the df
APOJ  is not in the df
HLA-DRB  is not in the df
BUB1B  is not in the df
BUB1  is not in the df
BRCA2  is not in the df
MUCI16  is not in the df
FBXW7  is not in the df
PMS2  is not in the df
POLE  is not in the df
RNF43  is not in the df
SPOP  is not in the df
FOXL2  is not in the df
CSMD3  is not in the df
USH2A  is not in the df
FAT3  is not in the df
FLG2  is not in the df
SI  is no

[['RYR2', 'KMT2C', 'LRP2', 'LRRK2', 'ZFHX4', 'FBXW7', 'POLE'],
 ['CSMD3',
  'USH2A',
  'FAT3',
  'FLG2',
  'SI',
  'MUC17',
  'FLG',
  'DNAH3',
  'DNAH5',
  'BRCA1',
  'TG',
  'DNAH10',
  'RELN',
  'DNAH8',
  'ADGRV1',
  'TENM1',
  'PKHD1',
  'RYR1',
  'CSMD1',
  'IGSF10',
  'HE4',
  'APOJ',
  'HLA-DRB',
  'BUB1B',
  'BUB1',
  'BRCA2',
  'MUCI16',
  'PMS2',
  'RNF43',
  'SPOP',
  'FOXL2']]

In [139]:
#ENSURE WE HAVE DATA FOR ALL
def inDF(genes, df):
    results=[]
    final=[]
    noinfo=[]
    
    for i in range(len(genes)):
        
        exists= genes[i] in df.columns
        
        if exists == 0:
            print(genes[i], " is not in the df")
            noinfo.append(genes[i])
        else:
            final.append(genes[i])
    
    results.append(final)
    results.append(noinfo)
    
    return results

In [149]:
#list2= smaller list!
def helper(list1, list2):
    newlist= list1.copy()
    
    for i in range(len(list2)):
        exist= newlist.count(list2[i])
        
        if exist!=0:
            continue
        
        print(list2[i], " added to new list")
        newlist.append(list2[i])
    
    return newlist

# Making new df w/ genes we will use in our model

In [150]:
#MAKE NEW DF W JUST COLUMNS FOR GENES WE WANT
newdf= proteins_masked.loc[: , LIST]

#NOT DON
scaler = StandardScaler()
scaled_data = scaler.fit_transform(proteins_masked)

#WHAT DO WE DO WITH IMPUTTING VALUES FOR 1 GENE LETS SAY

NameError: name 'LIST' is not defined

# Dimension reduction

In [151]:
#DIMENSION REDUCTION
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from hdbscan import HDBSCAN
from matplotlib import colormaps

In [155]:
#Dimension reduction analysis
#Evaluate performance of different dimension reduction approaches for our model.
#Based on this, we can see that UMAP is probably the best dimension reducer to use for our model 
#UMAP reduces the data and clusters are more visible than in others. It also shows linear trend that could be helpful in clustering 
plt.rcParams['figure.figsize'] = [12, 5]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(proteins_masked)

reducers = [PCA(), TSNE(), UMAP()]
reducers_names = ['PCA', 't-SNE', 'UMAP']

fig, axs = plt.subplots(1, 3, constrained_layout=True)

for i in range(3):
    embedding = reducers[i].fit_transform(scaled_data)
    x_vals = embedding[:, 0]
    y_vals = embedding[:, 1]
    axs[i].scatter(x_vals, y_vals, alpha=0.5)
    axs[i].set_title(reducers_names[i])
    

NameError: name 'StandardScaler' is not defined

In [153]:
reducer= UMAP()

# KMeans clustering model

In [154]:
embedding = reducer.fit_transform(scaled_data)
x_vals = embedding[:, 0]
y_vals = embedding[:, 1]

cluster_model = KMeans()

labels = cluster_model.fit_predict(embedding)
plt.scatter(x_vals, y_vals, c=labels, cmap='nipy_spectral', alpha=0.5)

NameError: name 'scaled_data' is not defined

# HDBSCAN clustering model

In [158]:
#CLUSTERING MODEL USING HDBSCAN 
plt.rcParams['figure.figsize'] = [12, 5]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(proteins_masked)

model = HDBSCAN()

reducers = [
    PCA(),
    TSNE(),
    UMAP()
]
reducers_names = ['PCA', 't-SNE', 'UMAP']

fig, axs = plt.subplots(1, 3, constrained_layout=True)

    
for i in range(3):
    embedding = reducers[i].fit_transform(scaled_data)
    x_vals = embedding[:, 0]
    y_vals = embedding[:, 1]
    labels = cluster_model.fit_predict(embedding)
    axs[i].scatter(x_vals, y_vals, c=labels, cmap='nipy_spectral', alpha=0.5)
    axs[i].set_title(reducers_names[i])



NameError: name 'StandardScaler' is not defined

In [ ]:
#ASSESSING MODELS